## Mod 7 Lecture 1:  Discriminative vs. Generative Models

**Do this assignment in Google Colab**

Upload the FHV_072023.csv dataset into the 'sample_data' folder in Google Colab.  Recall this data is from July 1, 2025 to July 15, 2025.

Reorient yourself to the data [HERE](https://data.cityofnewyork.us/Transportation/2023-High-Volume-FHV-Trip-Data/u253-aew4/about_data)

**We will only work with the first 5000 rows**


In [ ]:
# Imports
import pandas as pd 
import numpy as np

In [ ]:
#Only look at the first 5,000 rows for speed
df = pd.read_csv("None", nrows=5000)
df.head()

In [ ]:
df.info()

## Prompt 1 (Using AI for Entire Workflow) (We Do)

Copy and paste Prompt 1 into Gemini (within Google Colab) and paste the output below.  Just accept the output as is and run it step-by-step (or autorun it)

**It may take awhile to run!** 
*While it is running discuss*: 
- What do you expect the AI output to be (how many lines of code, all coding cells or mix of code and markdown cells, etc.)? 
- Is there any part of this prompt you would change (delete or add)?
- What parts of this prompt would you want to evaluate the output of first?
- Do you think everyone will get the same output?

“**Context**: I’m using the first 5,000 rows of a single NYC HVFHV CSVs with data dated for July 1, 2023 and July 15, 2023. I want an MLR predicting total_amount from trip_miles, trip_time, and engineered features. 

**Artifacts**: Produce a single Python code cell that: (1) reads one local CSVs I’ll upload; (2) selects columns: pickup_datetime, trip_miles, trip_time, base_passenger_fare, tips, total_amount; (3) concatenates; (4) drops rows with missing in these columns; (5) builds a Pipeline with imputer→scaler→LinearRegression; (6) fits train/test (80/20, random_state=42); (7) prints MAE/RMSE. 

**Rules**: No network calls, use pandas/sklearn only, don’t rename columns. 

**Evaluation**: Add 3 asserts (no NA, >1,000 rows, numeric dtypes) and print shapes.”

## Prompt 2 (Using AI for Feature Engineering) - You Do

Copy and paste Prompt 2 into Gemini (within Google Colab) and paste the output below this cell

“Suggest 6 feature ideas for predicting total_amount using HVFHV trips with columns: trip_miles, trip_time, base_passenger_fare, tips, total_amount. Label each as: linear, interaction, polynomial, or bucket. Explain expected direction. Output as a markdown table; no code.”

## Prompt 3 (Using AI for Diagnostics) - You Do 

Copy and paste Prompt 2 into Gemini (within Google Colab) and paste the output below this cell. Just accept the output as is and run it step-by-step (or autorun it)

“Generate a Python cell (no external calls) to: (1) plot residuals vs predicted for my fitted pipe and X_test, y_test; (2) print top 10 coefficients with feature names from pipe.named_steps["pre"].get_feature_names_out() aligned to pipe.named_steps["reg"].coef_; (3) sort by absolute value; (4) warn if any absolute residual > 3×RMSE.”

## Reflection (Share Out)

1) What prompt(s) gave you the "best" output 
2) What were some issues you saw in the prompt output? 
3) **How did the MLR model GenAI gave you compare to the model you did in Mod6L10/Mod6L11?** VERY IMPORTANT
4) If you all didn't get the same responses, what do you think is a limitation of GenAI? (think "memory")
5) Are there any ethical issues with the prompt output?